# wav file 처리 테스트


In [1]:
'''
# 라이브러리 정의
 - 시스템 라이브러리
 - 오디오 관련 라이브러리
 - 연산 및 Plot 관련 라이브러리 
 - 샤용자 라이브러리
'''

# Import Systems 
import struct
import io
import os
import sys
import time

# Import Audio
import wave
import pyaudio
#import librosa

import numpy as np
from scipy.io import wavfile
import scipy.io
import matplotlib.pyplot as plt

# User Libraries
import pyOssWavfile
import pyRoomAcoustic



# 폴더 이름 설정
 - 현재 프로젝트가 있는 폴더의 경로명을 설정
 - 프로젝트 내에 Audio File이 있는 폴더명을 프로젝트 경로와 합쳐 String으로 저장

# 오디오 파일 이름 설정
 - 불러 올 오디오 파일이름과 앞서 설정한 폴더 이름을 1개의 String으로 저장
   

In [2]:
# Path of audiofiles folder
audio_dir = os.path.join(os.getcwd(), 'audiofiles')

#print(audio_dir)


In [3]:
# Define wave file name text to open
# Path + wave file name

wav_fname = os.path.join(audio_dir, 'TyndallBruceMonument.wav')

#wav_fname = os.path.join(audio_dir, 'ElvedenHallMarbleHall.wav')
#wav_fname = os.path.join(audio_dir, 'EmptyApartmentBedroom.wav')
#wav_fname = os.path.join(audio_dir, 'FalklandPalaceRoyalTennisCourt.wav')
#wav_fname = os.path.join(audio_dir, 'InsidePiano.wav')
#wav_fname = os.path.join(audio_dir, 'MaesHowe.wav')
#wav_fname = os.path.join(audio_dir, 'SportsCentreUniversityOfYork.wav')
#wav_fname = os.path.join(audio_dir, 'StairwayUniversityOfYork.wav')
#wav_fname = os.path.join(audio_dir, 'StAndrewsChurch.wav')



# Read wav file using module scipy.io.wavflie
- Read wav file
- Plot data

In [ ]:

#wav_samplerate, wav_data = wavfile.read("audiofiles\TyndallBruceMonument.wav")

wav_samplerate, wav_data = wavfile.read(wav_fname)

time = wav_data.shape[0] / wav_samplerate

print(f"Samplerate = {wav_samplerate}")
print(f"Number of Channels = {wav_data.shape[1]}")
print(f"Length = {wav_data.shape[0]}")
print(f"Time = {time} sec")



In [ ]:
# Plot loaded wave file

plot_time = np.linspace(0., time, wav_data.shape[0])

plt.plot(plot_time, wav_data[:, 0], label="Left channel")
plt.plot(plot_time, wav_data[:, 1], label="Right channel")
plt.legend()
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.show()


In [ ]:
# save numpy data file

#np.save('ElvedenHallMarbleHall', wav_data)
#np.save(wav_fname, wav_data)


# numpy array 불러오기
- nubpy array로 저장한 오디오 데이터 파일 (.npy)를 불러오기
- 불러온 .npy파일을 plot하여 wav파일과 동일한 지 확인  

In [ ]:
# 
npy_fname = wav_fname + '.npy'

wav_narray = np.load(npy_fname)

#print(f"Samplerate = {wav_samplerate}")
print(f"Number of Channels = {wav_narray.shape[1]}")
print(f"Length = {wav_narray.shape[0]}")
#print(f"Time = {time} sec")



In [ ]:
# Plot npy array

plot_time = np.linspace(0., time, wav_narray.shape[0])

plt.plot(plot_time, wav_narray[:, 0], label="Left channel")
plt.plot(plot_time, wav_narray[:, 1], label="Right channel")
plt.legend()
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.show()


# Read wav file using module 
- using modified wav file read funcion
- extract wav file header information

In [6]:
# Read .wav file using pyOssWavfile.read() & Extract .wav file header information

ossWav_fmt_chunk, ossWav_data, ossWav_length = pyOssWavfile.read(wav_fname)

ossWav_format, ossWav_numch, ossWav_fs, ossWav_byterate, ossWav_blockalign, ossWav_bitdepth = pyOssWavfile.extractWavFmtChunk(ossWav_fmt_chunk)

print("Audio Format =", ossWav_format)
print("Number of Channel =", ossWav_numch)
print("Sampling Frequency =", ossWav_fs)
print("Byte Rate =", ossWav_byterate)       # 일종의 Checksum 
print("Block Align =", ossWav_blockalign)
print("Bits per Sample =", ossWav_bitdepth)





/Users/iMacDev/Documents/workspace_py/pyOssAudioAnalyzer/audiofiles/TyndallBruceMonument.wav
Audio Format = 3
Number of Channel = 2
Sampling Frequency = 44100
Byte Rate = 352800
Block Align = 8
Bits per Sample = 32
